# Введение
Описание задачи: ранжирование телепередач.
Идея для обучения модели: использовать данные за предыдущие недели для обучения модели, а для тестирования использовать данные за последнюю неделю.
Данные с Mediascope принять за эталонные.
Способ оценки модели: метрика mAP@k.

# Парсинг программы телепередач

К сожалению, Яндекс борется с парсингом программы телепередач с их сайта, поэтому в качестве источника телепрограммы я выбрал сайт ComboPlayer

In [68]:
import requests
from bs4 import BeautifulSoup


def get_program_for(date: str) -> dict[str, tuple[str, str]]:
    url = f'https://www.comboplayer.ru/tv-guide/{date}'
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, 'html.parser')

    table = soup.find('div', {'class': 'll-content'})
    channels = {}
    for tr in table.find_all('div', {'class': 'guide-channel'}):
        td_channel = tr.find('header', {'class': 'guide-channel__header'})
        if td_channel is not None:
            channel_name = td_channel.find('a').text.strip()
            programs = []
            for td_program in tr.find_all('li', {'class': 'programm-list__item'}):
                program_name = td_program.find('span', {'class': 'programm-list__title'}).text.strip()
                program_time = td_program.find('div', {'class': 'programm-list__time'}).text.strip()
                programs.append((program_name, program_time))
            channels[channel_name] = programs

    return channels



In [69]:
channels = get_program_for('today')
# Print the TV channel names and program schedules
for channel_name, programs in channels.items():
    print(channel_name)
    print('-' * len(channel_name))
    for program_name in programs:
        print(program_name)
    print()

Первый Канал
------------
('ПОДКАСТ.ЛАБ', '00:15')
('ПОДКАСТ.ЛАБ', '00:55')
('ПОДКАСТ.ЛАБ', '01:35')
('ПОДКАСТ.ЛАБ', '02:15')
('Новости', '03:00')
('ПОДКАСТ.ЛАБ', '03:05')
('ПОДКАСТ.ЛАБ', '03:45')
('ПОДКАСТ.ЛАБ', '04:20')
('Перерыв в вещании', '04:57')
('Телеканал "Доброе утро"', '05:00')
('Новости (с субтитрами)', '09:00')
('Премьера. АнтиФейк', '09:20')
('Жить здорово!', '09:55')
('Время покажет', '10:45')
('Новости (с субтитрами)', '12:00')
('Время покажет', '12:15')
('Большая игра', '14:10')
('Новости (с субтитрами)', '15:00')
('Время покажет', '15:15')
('Мужское / Женское', '16:00')
('Большая игра', '16:50')
('Вечерние новости (с субтитрами)', '18:00')
('Информационный канал', '18:20')
('Премьера. "Куклы наследника Тутти"', '20:00')
('Время', '21:00')
('Премьера. "Шпион"', '21:45')
('Большая игра', '22:45')
('Премьера. "Почка"', '23:45')
('ПОДКАСТ.ЛАБ', '00:15')
('ПОДКАСТ.ЛАБ', '00:55')
('ПОДКАСТ.ЛАБ', '01:35')
('ПОДКАСТ.ЛАБ', '02:15')
('Новости', '03:00')
('ПОДКАСТ.ЛАБ', '03:05')

# Парсинг данных Mediascope

In [19]:
def get_data_for(week: str = '20/03/2023 - 26/03/2023'):
    response = requests.post('https://mediascope.net/ajax/ratingsData.php', data={
        'arrFilter_pf[CITY]': 5096,
        'arrFilter_pf[PERIOD]': week,
        'arrFilter_pf[TARGET_AUDIENCE]': 'население 18-54',
        'arrFilter_pf[TYPE]': 43824,
        'set_filter': 'Y',
        'FILTER_TYPE': 'tv'
    })
    return response, response.headers, response.text